# Upskilling a no-skill classifier with Conformal Prediction

Whenever a scientist needs to build a model, they need to evaluate the results against certain metrics given some context.

This led to a culture that focus too much on optimizing metrics instead of measuring how the model would impact the business.

In this notebook I'll show a way to evaluate models through different lens by using conformal prediction that can helps you given a clear picture of what your model is predicting.

In [ ]:
import altair as alt
import numpy as np
import pandas as pd
from crepes import WrapClassifier
from sklearn.base import ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from utils.charts import display_static_altair_images
from utils.dataframes import make_classification_df


def calculate_coverage(
    X: pd.DataFrame,
    y: pd.Series,
    label: int,
    calibrated_conformal_classifier: WrapClassifier,
    alphas: list[float],
) -> pd.DataFrame:
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)

    sets = {
        alpha: calibrated_conformal_classifier.predict_set(X, confidence=1 - alpha)
        for alpha in alphas
    }

    count_true_label = sum(True for value in y if value == label)
    size = len(y)

    random_guessing = count_true_label / size

    results = []

    for alpha in alphas:
        count_coverage = 0
        count_sets = 0

        sets_aux = sets[alpha]
        for i, value in enumerate(y):
            if sets_aux[i, label]:
                count_sets += 1
                if value == label:
                    count_coverage += 1

        denominator_count_sets = count_sets if count_sets > 0 else 1

        res = {
            "alpha": alpha,
            "coverage": count_coverage,
            "% coverage (recall)": round(count_coverage * 100 / count_true_label, 2),
            "# sets containing target": count_sets,
            "% sets containing_target": round(count_sets * 100 / size, 2),
            "% sets correctly covering target (precision)": round(
                count_coverage * 100 / denominator_count_sets, 2
            ),
            "pp gain over random guessing": round(
                ((count_coverage * 100) / denominator_count_sets)
                - (random_guessing * 100),
                2,
            ),
        }

        results.append(res)

    return pd.DataFrame(results).set_index("alpha")


def train_and_calibrate(
    classifier: ClassifierMixin,
    X_train: pd.DataFrame,
    X_calib: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_calib: pd.Series,
    y_test: pd.Series,
) -> WrapClassifier:
    classifier.fit(X_train, y_train)
    y_preds = classifier.predict(X_test)

    print(classification_report(y_test, y_preds, zero_division=0))

    conformal_classifier = WrapClassifier(classifier)
    conformal_classifier.calibrate(
        X_calib.reset_index(drop=True), y_calib.reset_index(drop=True), class_cond=True
    )

    return conformal_classifier


start_alpha = 0.05
end_alpha = 0.95
num_alpha = int((end_alpha + start_alpha) / 0.05) - 1

alphas = np.linspace(start_alpha, end_alpha, num_alpha)
alphas = [round(i, 2) for i in alphas]

In [ ]:
features = ["a", "b"]
target = "y"

df_params = {
    "n_samples": [20000, 1000, 1000],
    "features": features,
    "centers": [(0, 0), (3, 0), (1, 2)],
    "cluster_std": [2, 0.5, 0.8],
    "random_state": 0,
}

df = make_classification_df(**df_params)

chart = (
    alt.Chart(df)
    .mark_point(size=10, opacity=0.5, filled=True)
    .encode(
        alt.X("a:Q").scale(domain=[-8, 8], clamp=True),
        alt.Y("b:Q").scale(domain=[-8, 8], clamp=True),
        alt.Color("y:N"),
    )
    .properties(
        width=600,
        height=600,
    )
)

display_static_altair_images(chart)

From the image above one can already imagine that it will be very hard to correctly classify the label 1 while being almost impossible to do the same with label 2.

Some would say "upsampling" while others would say "downsampling". Upsampling is objectively bad, I'm not wasting my time on why it is bad to create artificial data. Downsampling on the other hand is ok, but it adds an extra layer of complexity that can't be avoided if you want correct results.

For this case I prefer Conformal Prediction. It's a method that helps you say something like "in this region of the feature space we expect this probability for each label" with mathematical grounding.

What does this means in practice?
 * for a targeted campaign this could mean "anyone from this region is sure to be at least interested in this ad" (remember that are cases in which we don't even have the capability of attending the whole demand for something, so if we can reduce our spending in marketing campaigns while still guaranteeing we sell the whole stock it is a big win);
 * on the other hand, if we are talking about fraud detection, we can use a cheaper model to detect anyone slightly suspicious and then send the results to a more sofisticated and expensive model that maybe is a paid API we contracted for our business, which means that we don't waste too many resources trying to detect frauds on most transactions.

In [ ]:
X, y = df.filter(features), df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
X_calib, X_test, y_calib, y_test = train_test_split(
    X_test, y_test, test_size=0.5, random_state=0
)

train_data = (X_train, X_calib, X_test, y_train, y_calib, y_test)

new_df = make_classification_df(**{**df_params, "random_state": 1})
new_X, new_y = new_df.filter(features), new_df[target]

## Logistic Regression Classifier

Training a Logistic Regression Classifier yields a model that can't classify no one beyonds label 0 correctly. However, it isn't a useless model like the metrics make it seems so.

Looking at the table calculated on the results of each set whe can see how the model is able to perform at each alpha. This give us the ability to analyze where our model is performing correctly with virtually 100% certainty that the label is correct while showing us at which threshold the models starts to "fail".

That means even a classically bad classifier can be useful if we don't have anything better.

In [ ]:
conformal_logistic_regression_classifier = train_and_calibrate(
    LogisticRegression(random_state=0), *train_data
)
for label in (0, 1, 2):
    print(f"***** Results for label {label} *****")
    display(
        calculate_coverage(
            X_test,
            y_test,
            label,
            conformal_logistic_regression_classifier,
            alphas,
        )
    )

### Predicting on new data

It also keeps roughly the same results on data that follows the same distribution.

In [ ]:
for label in (0, 1, 2):
    print(f"***** Results for label {label} *****")
    display(
        calculate_coverage(
            new_X,
            new_y,
            label,
            conformal_logistic_regression_classifier,
            alphas,
        )
    )

## Random Forest Classifier

Of course the Random Forest Classifier would perform way better than a Logistic Regression Classifier. However it still performs very bad, but the Conformal Prediction on top of it still gives way better control over the results.

But hey, you do lose something in the label 0: there is no region with virtual 100% correct labels! There are some cases in which the no skill Logistic Regression Classifier can be more usefull than the smarter Random Forest Classifier, considering the results of the Conformal Prediction on top of it.

In [ ]:
conformal_random_forest_classifier = train_and_calibrate(
    RandomForestClassifier(random_state=0), *train_data
)
for label in (0, 1, 2):
    print(f"***** Results for label {label} *****")
    display(
        calculate_coverage(
            X_test,
            y_test,
            label,
            conformal_random_forest_classifier,
            alphas,
        )
    )

# Predicting on new data

Same as the logistic regression classifier, just so we can see it the conformal prediction helpings in many scenarios while confirming that we lost something for label 0 even in new data.

In [ ]:
for label in (0, 1, 2):
    print(f"***** Results for label {label} *****")
    display(
        calculate_coverage(
            new_X,
            new_y,
            label,
            conformal_random_forest_classifier,
            alphas,
        )
    )